<a href="https://colab.research.google.com/github/YellaCRE/Article_rec/blob/main/Article_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import os, re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#데이터 불러오기

In [ ]:
path = "/content/drive/MyDrive/황창조/C9/data/"
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
data = pd.read_csv(path + "articles_test_human.tsv", sep='\t')
#data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
data.shape

(500, 3)

In [ ]:
#int 변환
data['score_round'] = [int(round(i)) for i in data['score']]

In [ ]:
target = data[['score_round','sentence1']]
compared = data[['score_round','sentence2']]

In [ ]:
target.to_csv(path+"test_target.tsv",sep='\t',index = False)
compared.to_csv(path+"test_compared.tsv",sep='\t',index = False)

#토치세팅

In [ ]:
## CPU
device = torch.device("cpu")

## GPU
#device = torch.device("cuda:0")

In [ ]:
bert, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
dataset_target = nlp.data.TSVDataset(path + "test_target.tsv", field_indices=[1,0], num_discard_samples=1)
dataset_compared = nlp.data.TSVDataset(path + "test_compared.tsv", field_indices=[1,0], num_discard_samples=1)

#BERT토크나이져

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 1
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
def dataload(data):
  dataset = BERTDataset(data, 0, 1, tok, max_len, True, False)
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=5)
  return dataloader

In [ ]:
target_dataloader = dataload(dataset_target)
compared_dataloader = dataload(dataset_compared)

#BERT모델

In [ ]:
class BERTModel(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTModel, self).__init__()
        self.bert = bert

        #dropout
        self.dr_rate = dr_rate         
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        #embedding
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler

        return out

In [ ]:
model = BERTModel(bert).to(device)
model.eval()

In [ ]:
limit = 1000

target_output

In [ ]:
dataloader = target_dataloader
target_output=[]

for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length = valid_length
  pooled_output = model(token_ids, valid_length, segment_ids)
  
  target_output.append(pooled_output)
  batch_id += 1
  if batch_id >= limit:
    break

  0%|          | 0/500 [00:00<?, ?it/s]

compared_output

In [ ]:
dataloader = compared_dataloader 
compared_output=[]

for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length = valid_length
  pooled_output = model(token_ids, valid_length, segment_ids)
  
  compared_output.append(pooled_output)
  if batch_id >= limit:
    break

  0%|          | 0/500 [00:00<?, ?it/s]

#코사인유사도

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
cosim=[]
for i in range(0,len(target_output)):
  cosim.append(cosine_similarity(target_output[i].cpu().detach().numpy(),compared_output[i].cpu().detach().numpy())[0][0])

In [ ]:
y_pred=round(pd.Series(cosim)*5,1)
y_test=data['score'][:len(target_output)]
y_rand = [random.randint(0,5) for value in range(0,len(target_output))]

In [ ]:
data['pred_score'] = y_pred
data[['pred_score','score','sentence1','sentence2']].head(10)

,pred_score,score,sentence1,sentence2
0,5.0,1.40,비벤디 주식은 월요일에 3.6퍼센트 하락한 후 파리에서 15.80유로로 1.9% 하...,"뉴욕에서는, 비벤디 주식은 18.29달러로 1.4% 하락했다."
1,4.9,3.60,브레머는 한 가지 계획이 동네를 청소하고 지역 사회 프로젝트를 구축하기 위해 앞으로...,브레머는 이라크 전역의 이웃들을 청소하고 지역 사회 프로젝트를 건설하기 위해 앞으로...
2,4.7,3.50,그레이 데이비스 주지사는 어제 화재가 거의 20억 달러의 비용이 들 수 있다고 추정했다.,주 관리들은 그 비용을 거의 20억 달러로 추정했다.
3,4.6,3.20,American Express 회사는 필요한 500만 달러의 금액 보다 최소 300...,"시는 연방 기금을 요청했지만, American Express가 최소 300만 달러를..."
4,4.6,4.00,대서양 폭풍 시즌의 첫 번째 허리케인인 클로데트는 7월 15일 85mph의 바람으로...,대서양 폭풍 시즌의 첫 번째 허리케인인 클로데트는 7월 15일 텍사스 중부 해안을 ...
5,4.8,2.00,미국 2위 청량음료 제조사 펩시코 주식회사의 결과(nyse: pep- news-pe...,월스트리트도 알루미늄 제조사인 알코아 주식회사가( nyse : pep - news ...
6,4.7,3.60,테일러의 보좌관들은 갑작스러운 출발이 더 많은 혼란을 유발할 수 있다고 경고했다. ...,"그의 보좌관들은 갑작스러운 출발이 많은 유혈사태를 촉발시킬 수 있다고 경고했는데, ..."
7,4.4,4.20,그러나 오데뜨가 카리브해에서 처음으로 12월에 형성되었다고 센터는 말했다.,12월에 카리브해에서 처음으로 개발된 이름있는 폭풍이다.
8,4.9,4.00,AFL-CIO는 10월까지 후보자를 지지할지 결정하기 위해 기다리고 있다.,AFL-CIO는 수요일에 경선 전에 후보를 지지할 것인지 여부를 10월에 결정할 것...
9,4.9,3.25,"그것은 학계, 산업계, 정부, 그리고 대중의 300명 이상의 지도자들로부터 협의를 ...",NIH 로드맵이라고 불리는 이 계획은 산업계와 학계에서 300명 이상의 컨설턴트의 ...


In [ ]:
y_pred.describe()

count    500.000000
mean       3.519400
std        2.066248
min       -3.500000
25%        3.000000
50%        4.600000
75%        4.900000
max        5.000000
dtype: float64

#결과지표

In [ ]:
mean_absolute_error(y_test, y_pred)

1.9228100233829022

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

2.3620839627627825

마이너스값을 모두 0으로 치환

In [ ]:
y_pred_zero = np.where(y_pred < 0, 0, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred_zero)

1.7934100230938197

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred_zero))

2.17876291667956

랜덤값과 비교

In [ ]:
mean_absolute_error(y_test, y_rand)

1.7129979999999998

In [ ]:
np.sqrt(mean_squared_error(y_test, y_rand))

2.0870613182175553